# <center>World Map Geovisualization using Folium and Covid Data

In [2]:
import json
import folium
import requests
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize

In [3]:
conn= http.client.HTTPSConnection("api.covid19api.com")
payload=''
headers={}
conn.request("GET", "/summary",payload,headers)
res=conn.getresponse()
data=res.read().decode('UTF-8')


In [4]:
covid1=json.loads(data)

In [5]:
pd.json_normalize(covid1['Countries'], sep=",");

In [6]:
df=pd.DataFrame(covid1['Countries'])
df;

In [7]:
covid2=df.drop(columns=['CountryCode', 'Slug', 'Date', 'Premium'], axis=1)
covid2;

In [8]:
m=folium.Map(tiles='Stamen Terrain', min_zoom=1.5)
m;

### Obtain Geodata

geodata url: https://raw.githubusercontent.com/python-visualization/folium/master/examples/data

geo data file: world-countries.json

In [9]:
url='https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes=f'{url}/world-countries.json'

### Total Confirmed Covid Cases

In [10]:
folium.Choropleth(
    geo_data=country_shapes,
    min_zoom=2,
    name='Covid-19',
    data=covid2,
    columns=['Country', 'TotalConfirmed'],
    key_on ='feature.properties.name',
    fill_colors='OrRd',
    nan_fill_color='black',
    legend_name="Total Confirmed Covid Cases",
).add_to(m)
m;

In [11]:
covid2.update(covid2['TotalConfirmed'].map('Total Confirmed:{}'.format))
covid2.update(covid2['TotalRecovered'].map('Total Recoverd:{}'.format))

coordinates of countries: https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv

In [12]:
coordinates=pd.read_csv('https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv')
coordinates;

In [13]:
covid_final=pd.merge(covid2,coordinates,on='Country')

In [14]:
def plotDot(point):
    folium.CircleMarker(location=[point.latitude, point.longitude], 
                       radius=5,
                       weight=2,
                       popup=[point.Country, point.TotalConfirmed, point.TotalRecovered],
                       fill_color='#000000').add_to(m)

In [15]:
covid_final.apply(plotDot, axis=1)
m.fit_bounds(m.get_bounds())
m;

In [16]:
m1= folium.Map(tiles='StamenToner', min_zoom=2)
m1;

In [17]:
deaths=covid_final['TotalDeaths'].astype(float)


In [18]:
lat=covid_final['latitude'].astype(float)


In [19]:
lon=covid_final['longitude'].astype(float)

In [20]:
m1.add_child(HeatMap(zip(lat, lon,deaths), radius=0));

Country and Total Deaths from Covid19

In [21]:
def plotDot(point):
    folium.CircleMarker(location=[point.latitude,point.longitude],
                        radius=5,
                        weight=2,
                        popup=[point.Country,point.TotalDeaths],
                        fill_color='#000000').add_to(m1)
covid_final.apply(plotDot,axis=1)
m1.fit_bounds(m1.get_bounds())
m1